Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# 2. Model konwolucyjny (CNN)
Następnie sprawdzimy architekturę autokodera opartego na warstwach konwolucyjnych (CNN).

## 2.1. Architektura
Zapoznaj się z poniższymi implementacjami kodera i dekodera.

In [ ]:
from IPython.display import Code


Code(filename="src/cnn.py")

## 2.2. Uczenie modelu
Podobnie jak w przypadku poprzedniego przykładu, model został nauczony i załączony w repozytorium.

In [ ]:
LOG_PATH = "./data/CNN/logs"
CHECKPOINT_PATH = "./data/CNN/model.ckpt"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $LOG_PATH --host 0.0.0.0 --port 6006

In [ ]:
from src.dataset import SampledMNISTData
from src.cnn import CNNAutoencoder

Zakomentowany kod do uczenia modelu

W przypadku chęci modyfikacji, proszę odkomentować kod, przeuczyć model i zapisać go za pomocą metody `save_checkpoint`.

In [ ]:
# from lightning import Trainer
# from lightning.pytorch.loggers import TensorBoardLogger
# from src.dataset import SampledMNISTData

# conv_ae = CNNAutoencoder(latent_dim=2)
# trainer = Trainer(
#     max_epochs=10, 
#     logger=TensorBoardLogger(save_dir=LOG_PATH, name="conv_ae", default_hp_metric=False),
#     enable_checkpointing=False,
#     accelerator="auto",
# )
# mnist = SampledMNISTData(num_samples_per_class=-1)

# trainer.fit(
#     model=conv_ae,
#     train_dataloaders=mnist,
# )

# trainer.save_checkpoint(filepath=CHECKPOINT_PATH)

Wczytujemy przeuczony model:

In [ ]:
cnn_ae = CNNAutoencoder.load_from_checkpoint(checkpoint_path=CHECKPOINT_PATH, map_location="cpu")

## 2.3. Badanie jakości reprezentacji w zadaniu

Wczytujemy losową próbkę zbioru danych MNIST:

In [ ]:
mnist = SampledMNISTData(num_samples_per_class=100, seed=42)

Dla każdej próbki wyciągamy jej reprezentację (wektor dwu-wymiarowy) oraz etykietę:

In [ ]:
from src.utils import extract_representations


representations = extract_representations(
    model=cnn_ae,
    dataset=mnist,
)

In [ ]:
from src.utils import evaluate_linear    


auc_train, auc_test = evaluate_linear(
    z_train=representations["train"]["z"],
    y_train=representations["train"]["y"],
    z_test=representations["test"]["z"],
    y_test=representations["test"]["y"],
)

print(f"AUC => train: {auc_train * 100.0:.2f} [%], test: {auc_test * 100.0:.2f} [%]")

## 2.4. Wizualizacja przestrzeni reprezentacji

In [ ]:
%matplotlib inline

from src.utils import visualize_latent_spaces


visualize_latent_spaces(representations)

## 2.5. Wizualizacja jakości rekonstrukcji

In [ ]:
%matplotlib inline

from src.utils import visualize_random_sample

    
visualize_random_sample(model=cnn_ae, dataset=mnist.mnist_train)

## 2.6. Odporność modelu na przesunięcia

## Zadanie 2.1. (0 pkt)
Skopiuj implemetację funkcji `shift` z poprzedniego zeszytu do poniższej komórki.

In [ ]:
import torch


def shift(img: torch.Tensor, dx: int, dy: int) -> torch.Tensor:
    # TU WPISZ KOD
    raise NotImplementedError()

Wybieramy dowolną instancję ze zbioru danych:

In [ ]:
img, _ = mnist.mnist_train[256]

Zbadanie odporności na **przesunięcia poziome**:

In [ ]:
%matplotlib widget

from src.transforms import plot_transformation


def shift_horizontally(image: torch.Tensor, dx: int) -> torch.Tensor:
    return shift(img=image, dx=dx, dy=0)

    
plot_transformation(
    image=img,
    model=cnn_ae,
    transformation_fn=shift_horizontally,
    keep_channel_dim=True,  # Potrzebne do działania warstw kowolucyjnych
)

Zbadanie odporności na **przesunięcia pionowe**:

In [ ]:
def shift_vertically(image: torch.Tensor, dy: int) -> torch.Tensor:
    return shift(img=image, dx=0, dy=dy)


plot_transformation(
    image=img,
    model=cnn_ae,
    transformation_fn=shift_vertically,
    keep_channel_dim=True, # Potrzebne do działania warstw kowolucyjnych
)

**Obserwacja**: Widzimy, że dla małych wartości przesunięć model działa poprawnie (tzn. rekonstrukcja nadal przypomina tę samą cyfrę). Przy większych przesunięciach rekonstrukcja jest zaburzona, jednak przetransformowana cyfra jest również zbyt zaburzona przez przesunięcie.

## 2.7. Odporność na obroty
Inną ważną grupą symetrii w przypadku przetwarzania obrazów jest **grupa obrotów**, np. grupa P4 opisująca obroty obiektów o 90 stopni.

Zbadamy teraz jak się zachowuje model oparty o sieci konwolucyjne w przypadku obrotów.

## Zadanie 2.2. (2 pkt)
Zaimplementuj funkcję `rotate`, która obróci obraz wejściowy `img` o `angle` stopni w lewo względem środka obrazu. Funkcja powinna obsługiwać ujemne wartości parametru obrotu, tzn. `angle < 0` (obrót w prawo).

In [ ]:
import torch


def rotate(img: torch.Tensor, angle: float) -> torch.Tensor:
    # TU WPISZ KOD
    raise NotImplementedError()

In [ ]:
from src.transforms import plot_transformation


plot_transformation(
    image=img,
    model=cnn_ae,
    transformation_fn=rotate,
    min_param_value=0,
    max_param_value=360,
    step=10,
    keep_channel_dim=True,
)

## Zadanie 2.3 (0.5 pkt)
Czy autokoder CNN jest odporny na obroty obiektów? Z czego może to wynikać?

TU WPISZ ODPOWIEDŹ